# CurrDev --> Slides code, not a student notebook

### Import the notebook variables 

In [ ]:
%store -r MY_DB_NAME
%store -r MY_YB_PATH
%store -r MY_GITPOD_WORKSPACE_URL
%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03
%store -r MY_NOTEBOOK_DIR
%store -r MY_TSERVER_WEBSERVER_PORT
%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_DATA_DDL_FILE
%store -r MY_DATA_DML_FILE

# new for table id

DDL and DML

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_NOTEBOOK_DATA_FOLDER" "$MY_DATA_DDL_FILE" "$MY_DATA_DML_FILE"   
# Wishlist
YB_PATH=${1}
DB_NAME=${2}
DATA_FOLDER=${3}
DATA_DDL_FILE=${4}
DATA_DML_FILE=${5}

WISHLIST_DDL_PATH=${DATA_FOLDER}/${DATA_DDL_FILE}
WISHLIST_DML_PATH=${DATA_FOLDER}/${DATA_DML_FILE}
echo $WISHLIST_DDL_PATH
echo $WISHLIST_DML_PATH
cd $YB_PATH

# DDL file
./bin/ycqlsh -k ${DB_NAME} -f ${WISHLIST_DDL_PATH} 
sleep 1;

# # DML file
./bin/ycqlsh -k ${DB_NAME} -f ${WISHLIST_DML_PATH} 
sleep 1;


In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_NOTEBOOK_DATA_FOLDER" "$MY_DATA_DDL_FILE" "$MY_DATA_DML_FILE"   
# Wishlist
YB_PATH=${1}
DB_NAME=${2}
cd $YB_PATH

# # Describe relations
./bin/ycqlsh --execute "
  DESCRIBE tables;
"

###  TODO UPDATE LIKE INDEX
Create SST files

In order to read the actual data, we have to ask DocDB to (prematurely) persist the memtable so we can read the rocksdb contents. This is done using a yb-admin command. The above --print-entities command prints the table id (please mind to use the table id, lots of other objects in YugabyteDB have id's too!), which can be used to flush the tablet data to disk:

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  "$MY_HOST_IPv4_02" "$MY_TABLE_ID" # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}

TABLE_ID=${4}

echo $TABLE_ID
cd $YB_PATH

#yb-admin -init_master_addrs localhost:7100 flush_table_by_id 000033e8000030008000000000004232 600
./bin/yb-admin -init_master_addrs ${HOST_IPv4}:7100 flush_table_by_id ${TABLE_ID} 600


### View files

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  "$MY_HOST_IPv4_02" "$MY_TABLE_ID" "$MY_TABLET_ID" # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}
TABLE_ID=${4}
TABLET_ID=${5}


cd $YB_PATH
cd bin/

TABLE_ID_PATH=/Users/seth/yugabyte-data/node-1/disk-1/yb-data/tserver/data/rocksdb/table-${TABLE_ID}/tablet-${TABLET_ID=}

# cd $TABLE_ID_PATH

./sst_dump --command=scan --file=${TABLE_ID_PATH} --output_format=decoded_regulardb


## Index SST

In [ ]:
%%bash -s "$MY_YB_PATH" 
YB_PATH=${1}
cd $YB_PATH

# Verfiy the index has been created for the tbl_products_by_category
./bin/ycqlsh --execute "
  select * from ks_ybu.tbl_products_by_name;
"

# Examples

## q1 - Seq Scan


In [ ]:
%%bash -s "$MY_YB_PATH" 
YB_PATH=${1}
cd $YB_PATH


./bin/ycqlsh --execute "
 select product_name, product_id, price, description 
 from ks_ybu.tbl_products_by_category 
 where brand = 'Fern';
"

In [ ]:
%%bash -s "$MY_YB_PATH" 
YB_PATH=${1}
cd $YB_PATH


./bin/ycqlsh --execute "
 explain select product_name, product_id, price, description 
 from ks_ybu.tbl_products_by_category 
 where brand = ?;
"

## q2 - Range Scan using Partition Key

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Query plan: Sequential Scan
./bin/ycqlsh --execute "
  explain select product_name, product_id, price, description 
  from ks_ybu.tbl_products_by_category
  where category='Kids';
"  

## q3 - Sequence Scan

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Query plan: Sequential Scan
./bin/ycqlsh --execute "
  select product_name, product_id, price, description 
  from ks_ybu.tbl_products_by_category
  where brand = 'Fern';
"  

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Query plan: Sequential Scan
./bin/ycqlsh --execute "
  explain select product_name, product_id, price, description 
  from ks_ybu.tbl_products_by_category
  where brand = ?;
"  

## q4 - PK lookup

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Query plan: Sequential Scan
./bin/ycqlsh --execute "
  explain select product_name, product_id, price, description 
  from ks_ybu.tbl_products_by_category
  where category = ? and product_name = ? and product_id = ?;
"  

## q5 - Range Scan


In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH


./bin/ycqlsh --execute "
  explain select product_name, product_id, price, description 
  from ks_ybu.tbl_products_by_category
  where category = ?;
"  

## q6 - create index, product by brand

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Query plan: Sequential Scan
./bin/ycqlsh --execute "
  drop index if exists ks_ybu.idx_products_by_brand;
"  

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Query plan: Sequential Scan
./bin/ycqlsh --execute "
  create index idx_products_by_brand
  on ks_ybu.tbl_products_by_category
  (brand);
"  

### q6 - Describe index to show it

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
 describe ks_ybu.idx_products_by_brand
"    

### q6 - Explain --> Index Scan
Does not include price and description


In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
  explain select product_name, product_id, price, description 
  from ks_ybu.tbl_products_by_category 
  where brand = ?;
"    

### q6 Index SST Files
	

In [ ]:
MY_INDEX_TABLE_ID='6db1c23348dd49e4a7eae9c43725035e'
MY_INDEX_TABLET_ID='00d9931ef32945ccb1c17e1ca30ac001'
	
%store MY_INDEX_TABLE_ID
%store MY_INDEX_TABLET_ID

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  "$MY_HOST_IPv4_02" "$MY_INDEX_TABLE_ID" # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}

INDEX_TABLE_ID=${4}

echo $INDEX_TABLE_ID
cd $YB_PATH

#yb-admin -init_master_addrs localhost:7100 flush_table_by_id 000033e8000030008000000000004232 600
./bin/yb-admin -init_master_addrs ${HOST_IPv4}:7100 flush_table_by_id ${INDEX_TABLE_ID} 600


In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  "$MY_HOST_IPv4_02" "$MY_INDEX_TABLE_ID" "$MY_INDEX_TABLET_ID" # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}
INDEX_TABLE_ID=${4}
INDEX_TABLET_ID=${5}


cd $YB_PATH
cd bin/

TABLE_ID_PATH=/Users/seth/yugabyte-data/node-1/disk-1/yb-data/tserver/data/rocksdb/table-${INDEX_TABLE_ID}/tablet-${INDEX_TABLET_ID}
ls ${TABLE_ID_PATH}


./sst_dump --command=scan --file=${TABLE_ID_PATH} --output_format=decoded_regulardb
# ./sst_dump --command=scan --file=${TABLE_ID_PATH} --show_properties
# -command=scan --read_num=5

## q7  -- Index Only

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH


./bin/ycqlsh --execute "
  drop index  if exists ks_ybu.idx_products_by_brand_include;
"  


In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Query plan: Sequential Scan
./bin/ycqlsh --execute "
  create index idx_products_by_brand_include on ks_ybu.tbl_products_by_category
  (brand)
  include (price, description);
"  

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
 explain select product_name, product_id, price, description 
  from ks_ybu.tbl_products_by_category
  where brand =  ?;
"    

### q7a

In [ ]:
MY_INDEX_TABLE_ID='c84bdb1d1ab749939a03d758c6413f21'
MY_INDEX_TABLET_ID='318f1505ec15412fa76c409f81aacc38'
%store MY_INDEX_TABLE_ID
%store MY_INDEX_TABLET_ID

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  "$MY_HOST_IPv4_02" "$MY_INDEX_TABLE_ID" # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}

INDEX_TABLE_ID=${4}

echo $INDEX_TABLE_ID
cd $YB_PATH

#yb-admin -init_master_addrs localhost:7100 flush_table_by_id 000033e8000030008000000000004232 600
./bin/yb-admin -init_master_addrs ${HOST_IPv4}:7100 flush_table_by_id ${INDEX_TABLE_ID} 600


In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  "$MY_HOST_IPv4_02" "$MY_INDEX_TABLE_ID" "$MY_INDEX_TABLET_ID" # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}
INDEX_TABLE_ID=${4}
INDEX_TABLET_ID=${5}


cd $YB_PATH
cd bin/

TABLE_ID_PATH=/Users/seth/yugabyte-data/node-1/disk-1/yb-data/tserver/data/rocksdb/table-${INDEX_TABLE_ID}/tablet-${INDEX_TABLET_ID}
ls ${TABLE_ID_PATH}


./sst_dump --command=scan --file=${TABLE_ID_PATH} --output_format=decoded_regulardb
# ./sst_dump --command=scan --file=${TABLE_ID_PATH} --show_properties
# -command=scan --read_num=5

## q8 unique
uses gtin

### q8a
unique with constraint


In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "
  drop index if exists ks_ybu.idx_products_unq_gtin;
"  

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Query plan: Sequential Scan
./bin/ycqlsh --execute "
  create unique index idx_products_unq_gtin on ks_ybu.tbl_products_by_category
  (gtin)
"  

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
desc ks_ybu.idx_products_unq_gtin;
"    

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
 explain select product_name, product_id, price, description
  from ks_ybu.tbl_products_by_category
  where gtin = ?;
"    

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
 select product_name, product_id, price, description, brand, category, gtin
  from ks_ybu.tbl_products_by_category
  where gtin ='005456116545';
"    

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
insert into ks_ybu.tbl_products_by_category (category, product_name, product_id, brand, price, discount, description, gtin) values ('H20','Talc 6',62364,'Yeah',11.99,7,'1.6 liter ','006236226326');
"    

### Decode sst files by tablets
You need to change the host IPS.

In [ ]:
MY_OBJECT_NAME="idx_products_unq_gtin"
%store MY_OBJECT_NAME
print(MY_OBJECT_NAME)

In [ ]:
%%bash -s "$MY_HOST_IPv4_01" "$MY_HOST_IPv4_02" "$MY_HOST_IPv4_03" --out MY_HOST_IPv4

HOST_IPv4_01=${1}
HOST_IPv4_02=${2}
HOST_IPv4_03=${3}

# change the hosts for different tablet leaders
MY_HOST_IPv4=$HOST_IPv4_03

echo ${MY_HOST_IPv4}

Store the host ip for this procedure

In [ ]:
%store MY_HOST_IPv4
print(MY_HOST_IPv4)

Gets the table_ID

In [ ]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4" --out MY_INDEX_TABLE_ID

OBJECT_NAME=${1}
HOST_IPv4=${2}

MY_INDEX_TABLE_ID=`curl -s http://${HOST_IPv4}:8200/metrics | jq -r 'limit(1;  .[] | select(.attributes.namespace_name=="ks_ybu" and .type=="tablet" and .attributes.table_name=="'$OBJECT_NAME'") |  .attributes.table_id) '`

echo ${MY_INDEX_TABLE_ID}

Stores the table id

In [ ]:
%store MY_INDEX_TABLE_ID
print(MY_INDEX_TABLE_ID)

Gets the tablet_id leader for the host ip. This will change per host ip.

In [ ]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4" --out MY_INDEX_TABLET_ID
OBJECT_NAME=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")

MY_URL="http://${HOST_IPv4}:8200/metrics"

INDEX_TABLET_ID=`curl -s $MY_URL | jq --raw-output ' .[] | select(.attributes.namespace_name=="ks_ybu" and .type=="tablet" and .attributes.table_name=="idx_products_unq_gtin" ) | {tablet_id: .id, metrics: .metrics[] | select(.name == ("is_raft_leader") ) | select(.value == 1) } | select(.tablet_id) | {tablet_id} | .tablet_id '`

echo ${INDEX_TABLET_ID}

Stores the tablet leader id 

In [ ]:
%store MY_INDEX_TABLET_ID
print(MY_INDEX_TABLET_ID)

Flush the sst files for the tablet

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_HOST_IPv4_01" "$MY_INDEX_TABLE_ID"  # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}

INDEX_TABLE_ID=$( echo "${4}" | tr -d " ")

cd $YB_PATH

./bin/yb-admin -init_master_addrs ${HOST_IPv4}:7100 flush_table_by_id ${INDEX_TABLE_ID} 600


Decode the SST files for the tablet

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_HOST_IPv4_01" "$MY_INDEX_TABLE_ID" "$MY_INDEX_TABLET_ID" # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}
INDEX_TABLE_ID=$( echo "${4}" | tr -d " ")
INDEX_TABLET_ID=$( echo "${5}" | tr -d " ")

cd $YB_PATH
cd bin/

TABLE_ID_PATH=/Users/seth/yugabyte-data/node-1/disk-1/yb-data/tserver/data/rocksdb/table-${INDEX_TABLE_ID}/tablet-${INDEX_TABLET_ID}
ls -l  ${TABLE_ID_PATH}

./sst_dump --command=scan --file=${TABLE_ID_PATH} --output_format=decoded_regulardb

# ./sst_dump --command=scan --file=${TABLE_ID_PATH} --show_properties
# -command=scan --read_num=5

## q8b unique covering
uses gtin

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH


./bin/ycqlsh --execute "
  drop index if exists ks_ybu.idx_products_unq_gtin_inc;
"  


In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Query plan: Sequential Scan
./bin/ycqlsh --execute "
  create unique index idx_products_unq_gtin_inc on ks_ybu.tbl_products_by_category
  (gtin)
  include (brand, description, price)
"  

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
desc ks_ybu.idx_products_unq_gtin_inc;
"    

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
 explain select category, product_name, product_id, brand, description, price
  from ks_ybu.tbl_products_by_category
  where gtin = ?;
"    

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
 select category, product_name, product_id, brand, description, price
  from ks_ybu.tbl_products_by_category
  where gtin = '005456116545';
"    

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
insert into ks_ybu.tbl_products_by_category (category, product_name, product_id, brand, price, discount, description, gtin) values ('H20','Talc 6',62364,'Yeah',11.99,7,'1.6 liter ','006236226326');
"    

### q8b SST dump and decode

In [ ]:
MY_OBJECT_NAME="idx_products_unq_gtin"
%store MY_OBJECT_NAME
print(MY_OBJECT_NAME)

In [ ]:
%%bash -s "$MY_HOST_IPv4_01" "$MY_HOST_IPv4_02" "$MY_HOST_IPv4_03" --out MY_HOST_IPv4

HOST_IPv4_01=${1}
HOST_IPv4_02=${2}
HOST_IPv4_03=${3}

# change the hosts for different tablet leaders
MY_HOST_IPv4=$HOST_IPv4_02

echo ${MY_HOST_IPv4}

In [ ]:
%store MY_HOST_IPv4
print(MY_HOST_IPv4)


In [ ]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4" --out MY_INDEX_TABLE_ID

OBJECT_NAME=${1}
HOST_IPv4=${2}

MY_INDEX_TABLE_ID=`curl -s http://${HOST_IPv4}:8200/metrics | jq -r 'limit(1;  .[] | select(.attributes.namespace_name=="ks_ybu" and .type=="tablet" and .attributes.table_name=="'$OBJECT_NAME'") |  .attributes.table_id) '`

echo ${MY_INDEX_TABLE_ID} 


In [ ]:
%store MY_INDEX_TABLE_ID
print(MY_INDEX_TABLE_ID)

In [ ]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4" --out MY_INDEX_TABLET_ID
OBJECT_NAME=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")

MY_URL="http://${HOST_IPv4}:8200/metrics"

INDEX_TABLET_ID=`curl -s $MY_URL | jq --raw-output ' .[] | select(.attributes.namespace_name=="ks_ybu" and .type=="tablet" and .attributes.table_name=="idx_products_unq_gtin" ) | {tablet_id: .id, metrics: .metrics[] | select(.name == ("is_raft_leader") ) | select(.value == 1) } | select(.tablet_id) | {tablet_id} | .tablet_id '`

echo ${INDEX_TABLET_ID}

In [ ]:
%store MY_INDEX_TABLET_ID
print(MY_INDEX_TABLET_ID)

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_HOST_IPv4_01" "$MY_INDEX_TABLE_ID"  # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}

INDEX_TABLE_ID=$( echo "${4}" | tr -d " ")

cd $YB_PATH

./bin/yb-admin -init_master_addrs ${HOST_IPv4}:7100 flush_table_by_id ${INDEX_TABLE_ID} 600


In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_HOST_IPv4_01" "$MY_INDEX_TABLE_ID" "$MY_INDEX_TABLET_ID" # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}
INDEX_TABLE_ID=$( echo "${4}" | tr -d " ")
INDEX_TABLET_ID=$( echo "${5}" | tr -d " ")

cd $YB_PATH
cd bin/

TABLE_ID_PATH=/Users/seth/yugabyte-data/node-1/disk-1/yb-data/tserver/data/rocksdb/table-${INDEX_TABLE_ID}/tablet-${INDEX_TABLET_ID}
ls -l  ${TABLE_ID_PATH}

./sst_dump --command=scan --file=${TABLE_ID_PATH} --output_format=decoded_regulardb


## q9 

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "
  drop index if exists ks_ybu.idx_products_high_discount;
"  

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Query plan: Sequential Scan
./bin/ycqlsh --execute "
  create index idx_products_high_discount on ks_ybu.tbl_products_by_category
  ( discount )
  include ( brand, description, price )
  where discount > 10;
"  

### q9a partial

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
desc ks_ybu.idx_products_high_discount;
"    

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
explain select category, product_name, product_id, brand, description, price, discount
  from ks_ybu.tbl_products_by_category
  where discount > 10;
"    

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
 select category, product_name, product_id, brand, description, price, discount
  from ks_ybu.tbl_products_by_category
  where discount >10;
"    

### q9a
Partial index SST dump and decode


In [ ]:
MY_OBJECT_NAME="idx_products_high_discount"
%store MY_OBJECT_NAME
print(MY_OBJECT_NAME)

In [ ]:
%%bash -s "$MY_HOST_IPv4_01" "$MY_HOST_IPv4_02" "$MY_HOST_IPv4_03" --out MY_HOST_IPv4

HOST_IPv4_01=${1}
HOST_IPv4_02=${2}
HOST_IPv4_03=${3}

# change the hosts for different tablet leaders
#MY_HOST_IPv4=$HOST_IPv4_01
MY_HOST_IPv4=$HOST_IPv4_02
#MY_HOST_IPv4=$HOST_IPv4_03

echo ${MY_HOST_IPv4}

In [ ]:
%store MY_HOST_IPv4
print(MY_HOST_IPv4)


In [ ]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4" --out MY_INDEX_TABLE_ID

OBJECT_NAME=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")

MY_URL="http://${HOST_IPv4}:8200/metrics"

MY_INDEX_TABLE_ID=`curl -s ${MY_URL} | jq -r 'limit(1;  .[] | select(.attributes.namespace_name=="ks_ybu" and .type=="tablet" and .attributes.table_name=="'$OBJECT_NAME'") |  .attributes.table_id) '`

echo ${MY_INDEX_TABLE_ID} 


In [ ]:
%store MY_INDEX_TABLE_ID
print(MY_INDEX_TABLE_ID)

In [ ]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4" --out MY_INDEX_TABLET_ID
OBJECT_NAME=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")

MY_URL="http://${HOST_IPv4}:8200/metrics"


INDEX_TABLET_ID=`curl -s ${MY_URL} | jq --raw-output ' .[] | select(.attributes.namespace_name=="ks_ybu" and .type=="tablet" and .attributes.table_name=="'$OBJECT_NAME'") | {tablet_id: .id, metrics: .metrics[] | select(.name == ("is_raft_leader") ) | select(.value == 1) } | select(.tablet_id) | {tablet_id} | .tablet_id '`

echo ${INDEX_TABLET_ID}

In [ ]:
%store MY_INDEX_TABLET_ID
print(MY_INDEX_TABLET_ID)

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_HOST_IPv4_01" "$MY_INDEX_TABLE_ID"  # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}

INDEX_TABLE_ID=$( echo "${4}" | tr -d " ")

cd $YB_PATH

./bin/yb-admin -init_master_addrs ${HOST_IPv4}:7100 flush_table_by_id ${INDEX_TABLE_ID} 600


In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_HOST_IPv4_01" "$MY_INDEX_TABLE_ID" "$MY_INDEX_TABLET_ID" # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}
INDEX_TABLE_ID=$( echo "${4}" | tr -d " ")
INDEX_TABLET_ID=$( echo "${5}" | tr -d " ")

cd $YB_PATH
cd bin/

TABLE_ID_PATH=/Users/seth/yugabyte-data/node-1/disk-1/yb-data/tserver/data/rocksdb/table-${INDEX_TABLE_ID}/tablet-${INDEX_TABLET_ID}
ls -l  ${TABLE_ID_PATH}

./sst_dump --command=scan --file=${TABLE_ID_PATH} --output_format=decoded_regulardb

# ./sst_dump --command=scan --file=${TABLE_ID_PATH} --show_properties
# -command=scan --read_num=5

### q9b partial with not null

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
desc ks_ybu.idx_products_high_discount;
"    

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
select category, product_name, product_id, brand, description, price, discount
  from ks_ybu.tbl_products_by_category
  where discount IN (9,10)
"    

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
 explain select category, product_name, product_id, brand, description, price, discount
  from ks_ybu.tbl_products_by_category
  where discount > 11
"    

### q9b not null
Partial index SST dump and decode


In [ ]:
MY_OBJECT_NAME="idx_products_high_discount"
%store MY_OBJECT_NAME
print(MY_OBJECT_NAME)

In [ ]:
%%bash -s "$MY_HOST_IPv4_01" "$MY_HOST_IPv4_02" "$MY_HOST_IPv4_03" --out MY_HOST_IPv4

HOST_IPv4_01=${1}
HOST_IPv4_02=${2}
HOST_IPv4_03=${3}

# change the hosts for different tablet leaders
MY_HOST_IPv4=$HOST_IPv4_03

echo ${MY_HOST_IPv4}

In [ ]:
%store MY_HOST_IPv4
print(MY_HOST_IPv4)


In [ ]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4" --out MY_INDEX_TABLE_ID

OBJECT_NAME=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")

MY_URL="http://${HOST_IPv4}:8200/metrics"

MY_INDEX_TABLE_ID=`curl -s ${MY_URL} | jq -r 'limit(1;  .[] | select(.attributes.namespace_name=="ks_ybu" and .type=="tablet" and .attributes.table_name=="'$OBJECT_NAME'") |  .attributes.table_id) '`

echo ${MY_INDEX_TABLE_ID} 


In [ ]:
%store MY_INDEX_TABLE_ID
print(MY_INDEX_TABLE_ID)

In [ ]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4" --out MY_INDEX_TABLET_ID
OBJECT_NAME=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")

MY_URL="http://${HOST_IPv4}:8200/metrics"


INDEX_TABLET_ID=`curl -s ${MY_URL} | jq --raw-output ' .[] | select(.attributes.namespace_name=="ks_ybu" and .type=="tablet" and .attributes.table_name=="'$OBJECT_NAME'") | {tablet_id: .id, metrics: .metrics[] | select(.name == ("is_raft_leader") ) | select(.value == 1) } | select(.tablet_id) | {tablet_id} | .tablet_id '`

echo ${INDEX_TABLET_ID}

In [ ]:
%store MY_INDEX_TABLET_ID
print(MY_INDEX_TABLET_ID)

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_HOST_IPv4_01" "$MY_INDEX_TABLE_ID"  # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}

INDEX_TABLE_ID=$( echo "${4}" | tr -d " ")

cd $YB_PATH

./bin/yb-admin -init_master_addrs ${HOST_IPv4}:7100 flush_table_by_id ${INDEX_TABLE_ID} 600


In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_HOST_IPv4_01" "$MY_INDEX_TABLE_ID" "$MY_INDEX_TABLET_ID" # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}
HOST_IPv4=${3}
INDEX_TABLE_ID=$( echo "${4}" | tr -d " ")
INDEX_TABLET_ID=$( echo "${5}" | tr -d " ")

cd $YB_PATH
cd bin/

TABLE_ID_PATH=/Users/seth/yugabyte-data/node-1/disk-1/yb-data/tserver/data/rocksdb/table-${INDEX_TABLE_ID}/tablet-${INDEX_TABLET_ID}
ls -l  ${TABLE_ID_PATH}

./sst_dump --command=scan --file=${TABLE_ID_PATH} --output_format=decoded_regulardb

# ./sst_dump --command=scan --file=${TABLE_ID_PATH} --show_properties
# -command=scan --read_num=5

#### Query plan
In the preceding cell, the `WHERE` clause on line #13, searches for all rows that contain the product name "Hotdogs". The resulting query plan reveals that this query needed to scan every row in the primary table to determine which rows contained the product name "Hotdogs".

### Aggregate Scan
Run the following cell to determine the query plan for an query using an aggregate function. This query plan reveals another type of scan called a Range scan. A range scan reveals that many rows values are needed to satisfy this query.

In [ ]:
%%bash -s "$MY_YB_PATH" 
YB_PATH=${1}
cd $YB_PATH

# Query Plan for Aggregate function
./bin/ycqlsh --execute "
  explain select sum(quantity) as subtotal 
  from ks_ybu.tbl_products_by_wishlist where wishlist_id=?;
"  

### Secondary Indexes 
In this section, you will create an index and determine how this affects the efficiency of a query by comparing the query plans.

Run the following cell to create and describe an index by describing the tbl_products_by_category table.

In [ ]:
%%bash -s "$MY_YB_PATH" 
YB_PATH=${1}
cd $YB_PATH

# Create a secondary index, product_name
./bin/ycqlsh --execute "
  CREATE INDEX idx_products_by_name 
  ON ks_ybu.tbl_products_by_category (product_name) 
  INCLUDE (description);
"   

# Verfiy the index has been created for the tbl_products_by_category
./bin/ycqlsh --execute "
  DESC ks_ybu.tbl_products_by_category
"

#### Verify the query plan for a secondary index
In the previous cell, a secondary index was created so that the product name can be efficiently queried on the tbl_products_by_category table. The DESC keyword is used to verify if the index was created correctly. 

> **Important:** The attributes in the primary key remain a part of the secondary index and can be retrieved from the index without an extra trip to the primary table, improving operational efficiency. 

The description attribute was added using the `INCLUDE` keyword to the secondary index to create a covering index.

Run the following cell to verify if adding a secondary index for the `product_name` can make the product_name query on the `tbl_products_by_category` more efficient.

In [ ]:
%%bash -s "$MY_YB_PATH"   # Sequential Scan
YB_PATH=${1}
cd $YB_PATH

# Expose the query plan for a covering index
./bin/ycqlsh --execute "
  EXPLAIN SELECT product_name, category, price, description, product_id 
  FROM ks_ybu.tbl_products_by_category where product_name=?;
"  

#### Index Only Scan
By running the previous cell, you were able to determine that by adding a secondary index for the condition attribute, `product_name`, the query plan has changed from a sequential scan to an index only scan. This has improved this query's performance by orders of magnitude when accounting for large scale workloads. 

The `INCLUDE` clause plays an essential role in making queries more efficient and indexes more useful.  In the previous cell, the `product_name` was added to the index by using the `INCLUDE` clause. Now, the secondary index contains the values of the `product_name` attribute, creating a covering index, reducing a trip to the primary table, also known as the heap in PostgreSQL.

For example, if you had not added the `description` attribute in the `INCLUDE` clause, that would have made the query plan an index scan. This means that in our example, although the index was used to locate product_name, the primary table was still accessed to retrieve the data for the `description` column.

### Unique Indexes
You can disallow a column from having duplicate values by using a unique constraint as shown in the following cell.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
  drop index if exists ks_ybu.idx_unique_product_by_id;
"  
   

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
  create unique index idx_unique_product_by_id 
  on ks_ybu.tbl_products_by_category(product_id)
  include (price, description);
"  
   

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH


  
./bin/ycqlsh --execute "
 desc ks_ybu.idx_unique_product_by_id;
"  


In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH


# This statement will cause an error since duplicate values for the 
# product_id are not allowed, 87c7624a-4af5-4347-922d-ab43ab32476b.
./bin/ycqlsh --execute "
  INSERT INTO ks_ybu.tbl_products_by_category (
    product_name, 
    description, 
    price, 
    category,
    product_id 
  ) VALUES (
    'Guard dogs',
    'Doberman Pinchers',
    643.99,
    'Security',
    87c7624a-4af5-4347-922d-ab43ab32476b
  );
"

>**Important:** Notice the error in the result set: `Execution Error. Duplicate value disallowed by unique index idx_unique_product_by_id`. This verifies that the unique index that was created for the `product_id` is preventing a duplicate value for this attribute. Currently the same `product_id` identitifies a backpack and cannot be used again for a different product.

#### Partial Indexes

Use a range partition to reduce the amount of data that requires scanning.
In this index, you will partition all products over $30 in the Office Supply category. 

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
  CREATE INDEX ON ks_ybu.tbl_products_by_category(price) 
  INCLUDE (description, product_name) 
  WHERE price > 30 and category = 'Office Supplies';
"

./bin/ycqlsh --execute "
  DESC ks_ybu.tbl_products_by_category
"  

### Specific Indexes
An index can be created that specifies a particular value of an attribute. For instance, suppose that a popular item in our ecommerce site is searched for constantly. It may be beneficial to create an index that satisfies this query quickly and efficiently. In the following cell, there is an example of an index for a popular product, batteries.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Specific Index
./bin/ycqlsh --execute "
  CREATE INDEX idx_batteries_desc 
  ON ks_ybu.tbl_products_by_category (description) 
  INCLUDE (product_name) 
  WHERE description='Batteries';
"
#  Index Only Scan
./bin/ycqlsh --execute "
  EXPLAIN SELECT 
    description, 
    product_name, 
    product_id, 
    category, 
    price 
  FROM ks_ybu.tbl_products_by_category 
  WHERE description='Batteries';
"  

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

#  Seq Scan
./bin/ycqlsh --execute "
  EXPLAIN SELECT description, product_name, product_id, category, price FROM ks_ybu.tbl_products_by_category 
  WHERE description='Hotdogs';
"  

>**Important:** Note that in the previous cell, an index only scan will occur only if the WHERE clause predicate returns the same expression as originally stated in the index creation statement.

Run the following statement to verify that a query to a different expression will not use the index to satisfy the query.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

#  Seq Scan
./bin/ycqlsh --execute "
  EXPLAIN SELECT description, product_name, product_id, category, price FROM ks_ybu.tbl_products_by_category 
  WHERE description=?;
"  

#### Collections
More complex data structures allow YCQL to store data sets that offers more flexibility in its data model capabilities. Very important when a trying to reduce the amount of tables that need to be queried. Run the following cell to add a new column to the category table whose data type is an ordered list with text elements.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Collection: Create a column with a list containing text elements
./bin/ycqlsh --execute "
  ALTER TABLE ks_ybu.tbl_products_by_category 
  ADD warehouse_ids LIST<TEXT>;
"

./bin/ycqlsh --execute "
  DESC ks_ybu.tbl_products_by_category
"

#### Collection: SET
Run the following cell to create a new column on the category table. This new column will be a SET with text elements. 

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Collection: Create a column for a set containing text elements
./bin/ycqlsh --execute "
  ALTER TABLE ks_ybu.tbl_products_by_category 
  ADD tags SET<TEXT>;
"

./bin/ycqlsh --execute "
  DESC ks_ybu.tbl_products_by_category
"

#### Frozen Collections
Run the following cell to crate a frozen collection. In this example a list was frozen. While this value in this column can be dropped, its elements cannot be manipulated.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Frozen Collection
./bin/ycqlsh --execute "
  ALTER TABLE ks_ybu.tbl_products_by_category 
  ADD store_locations FROZEN<LIST<TEXT>>;
"

./bin/ycqlsh --execute "
  DESC ks_ybu.tbl_products_by_category
"

#### JSON

JSONB is considered the best way to utilize complex data structures since in YCQL, JSONB is searchable. This is not true of the other collections in YCQL. Also note that collections can be used in JSON as well. Run the following cell and note the pattern necessary to write a JSON object to a table in YCQL.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
  INSERT INTO ks_ybu.tbl_products_by_category (
    category, 
    price, 
    product_id, 
    sku_details
  ) VALUES (
    'Grocery',
    9.99,
    6eb8d774-8b03-4457-a8e9-710339ca7165,
    '{
      \"product_id\": \"6eb8d774-8b03-4457-a8e9-710339ca7165\",
      \"warehouse_sku\": \"8jk39d03-8b03-4457-a8e9-710339ca7165\"
    }'
  );
"

./bin/ycqlsh --execute "
  SELECT * FROM ks_ybu.tbl_products_by_category;
"

In the preceding cell, note the syntax necessary to insert a JSON object into the table. Outside the curly brackets are single quotes, where inside the brackets there are double quotes.

#### JSONB Index

JSONB is considered the best way to utilize complex data structures since in YCQL, JSONB is searchable.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
  CREATE INDEX idx_sku_details_name 
  ON ks_ybu.tbl_products_by_category((sku_details->>'Name'));
"

./bin/ycqlsh --execute "
  DESC  ks_ybu.tbl_products_by_category
"

./bin/ycqlsh --execute "
  EXPLAIN SELECT category, price, product_id FROM ks_ybu.tbl_products_by_category 
  WHERE (sku_details->>'Name')=?;
"

In the preceding cell, note the syntax that is required to create an index using a JSON key. Also note the syntax used to search by the key in a JSON object.

### Time to Live

YCQL offers data expiration. In the context of data modelling, removing "old" or deprecated data can improve database operational costs as well as storage costs.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
  CREATE TABLE ks_ybu.tbl_wishlists_by_user (
    user_id BIGINT, 
    todolist_name TEXT, 
    todolist_id UUID, 
    is_public BOOLEAN, 
    PRIMARY KEY((user_id), todolist_name)
  ) 
  WITH CLUSTERING ORDER BY (todolist_name DESC);
"

In the following cell, a row is added to the table with a time-to-live of 5 seconds. Once the time has expired, notice that the row is no longer in the table. This is a TTL scoped to the data row.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "
  INSERT INTO ks_ybu.tbl_wishlists_by_user(
      user_id,
      name,  
      wishlist_id,
      is_public
    ) VALUES (
      'Mark', 
      'Grocery', 
      2a70494e-6b68-4739-b3e0-ff06aa0a2d67, 
      true
    ) 
    USING TTL 5;
  "  

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "
  INSERT INTO ks_ybu.tbl_wishlists_by_user(
      user_id, 
      name, 
      wishlist_id,
      is_public
    ) VALUES (
      'Mark', 
      'Grocery', 
      2a70494e-6b68-4739-b3e0-ff06aa0a2d67, 
      true
    ) 
    USING TTL 5;
  "  

  ./bin/ycqlsh --execute "
    SELECT * FROM ks_ybu.tbl_wishlists_by_user;
  "  

Wait five seconds before running the following cell. This will verify that the row level data expiration property run successfully.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH
   
./bin/ycqlsh --execute "
  SELECT * FROM ks_ybu.tbl_wishlists_by_user;
" 

---


In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH
   
./bin/ycqlsh --execute "
 CREATE TABLE ks_ybu.music (
  artist TEXT,
  song TEXT,
  album TEXT,
  year INT,
  rating FLOAT,
  genre TEXT,
  PRIMARY KEY ((artist), year, song)
);
" 

TODO: Add to exercises for ORDER BY
Order by currently only support the ordering of columns following their declared order in the PRIMARY KEY

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH
   
./bin/ycqlsh --execute "
  SELECT * FROM ks_ybu.music where artist='Beattles' order by song;
" 

If the row is still visible, wait a few more seconds to run the preceding cell. This will verify that the row has expired as expected.